Group nr:

Name 1 and CID: Philip Fredriksson (Philiper)

Name 2 and CID: Abdulahman hameshli (Hameshli)

In [1]:
import numpy as np
import pygame
from IPython.display import Image
from BalanceRobot import BalanceRobot

pygame 2.1.2 (SDL 2.0.18, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Balance robot

In this assignment, we will perform parameter estimation on a balancing robot. First, we will show how the equations of motion can be derived, then we will run a control algorithm to balance the robot. While the robot is balancing we will collect data and using this data in a regression problem we will identify some physical properties of the robot. Below we can see a drawing of the robot.


<img src="img/BallanceRobot.png" width="300"/>

| Parameter | Description | To be estimated |
| --- | --- | --- |
| $m_1$ | Mass of main body | Yes |
| $l$ | Distance from wheel hub to $m_1$ | Yes |
| $I_1$ | Moment of inertia for main body | Yes |
| $m_2$ | Mass of wheels | No |
| $r$ | Radious of wheels | No |
| $I_2$ | Moment of inertia for of wheels | No |
| - | - | - |
| $\alpha_1$ | Angle of main body | |
| $\alpha_2$ | Angle of wheel from position 0 | |


## Equations of motion

Here we show one way to calculate the equations of motion for the system. 

We start with setting up the kinematic relationships. We define $(x_1, y_1)$ to be the position for the centre of mass of the main body and $x_2$ the position of the wheels, both with respect to the coordinate frame. The kinematics equations for the system then become:

$x_2 = - \alpha_2 r$

$x_1 = - \alpha_2 r - l sin(\alpha_1)$

$y_1 = r + l cos(\alpha_1)$

$\dot{x}_1 = - r \dot{\alpha}_2 - l cos(\alpha_1) \dot{\alpha_1}$

$\dot{y}_1 = -l sin(\alpha_1) \dot{\alpha}_1$

$\dot{x}_2 = - r \dot{\alpha}_2 $ 

Potential Energy for the sytstem:

$V = l cos(\alpha_1)m_1g$

Kinematic energy:

$T = \frac{1}{2} m_1 (\dot{x}^2_1 + \dot{y}^2_1) + \frac{1}{2} m_2 \dot{x}^2_2 + \frac{1}{2} I_1 \dot{\alpha}^2_1 + \frac{1}{2} I_2 \dot{\alpha}_2^2$

From these the equations of motion for the balance robot can be calculated, here we use the Euler Lagrange method. First the Lagrangian is calculated by: 

$L = T - V$

and becomes:

$L = \frac{1}{2} m_1 (r^2 \dot{\alpha}^2_2 + 2 r \dot{\alpha}_2 l cos(\alpha_1) \dot{\alpha_1} + l^2  \dot{\alpha}_1^2) +
\frac{1}{2} m_2 r^2 \dot{\alpha}_2^2 + \frac{1}{2} I_1 \dot{\alpha}^2_1 + \frac{1}{2} I_2 \dot{\alpha}_2^2 -l cos(\alpha_1)m_1g$

Euler Lagrange equation is defined by: 

$\frac{d}{dt}(\frac{\partial L }{\partial \dot{q}_i}) - \frac{\partial L }{\partial q_i} = Q_i$

Where $q_1 = \alpha_1$, $q_2 = \alpha_2$ and $Q_i$ reprsents external force and will be torque $M$ generated by the motors. 

The equations of motion after simplification becomes:

$M = m_1 r \ddot{\alpha}_2 l cos(\alpha_1) + m_1 l^2 \ddot{\alpha}_1 + I_1 \ddot{\alpha}_1  - lsin(\alpha_1)m_1g$

and

$-M =  m_1 r^2 \ddot{\alpha}_2 + m_1 rlcos(\alpha_1) \ddot{\alpha}_1 - m_1 rlsin(\alpha_1)\dot{\alpha}_1^2 + m_2 r^2 \ddot{\alpha}_2 + I_2 \ddot{\alpha}_2$

## Exersise 1:

To perform parameter identification with linear regression the equations need to be reformulated to the form $ y = X^T \theta $ where the vector $\theta$ will be estimated using regression. There are 3 parameters we want to estimate, $m_1$, $l$ and $I_1$, all other parameters can be assumed to be known. Rewrite the two equations of motion to the form $ y = X^T \theta $ so that $m_1$, $l$ and $I_1$ can be extracted from $\theta$. 

Hint: It might no be possible to linearize equations so that $\theta$ directly has the paramters $m_1$, $l$, $I_1$ but rather it might contain combinations of the parameters from which we later can calculate the values of $m_1$, $l$, $I_1$. 


Anwser:
$\left[ \begin{array}{c}
 M \\
 -M -I_2 \ddot{\alpha_2} - m_2r^2\ddot{\alpha_2}
\end{array} \right] = 
\left[ \begin{array}{ccc}
  \ddot{\alpha_1} &r\ddot{\alpha_2} cos(\alpha_1)-g\cdot sin(\alpha_1) & 0 \\
   0 & r\ddot{\alpha_1}\cdot cos(\alpha_1) - r\dot{\alpha_1}^2 \cdot sin(\alpha_1) & r^2\ddot{\alpha_2}  \end{array} \right]
\left[ \begin{array}{c}
 m_1l^2+I_1\\
  m_1l\\
  m_1
\end{array} \right]$



## Simulator

For those interested the equations of motion are not only interesing for the parameter identification but can also be used to build a simulator for the system. The equations of motion are rewritten to something that is easier to integrate.

$ \ddot{\alpha}_2   = \frac{-M -m_1 rlcos(\alpha_1) (M + lm_1gsin(\alpha_1)) / (m_1 l^2 + I_1) + m_1 rlsin(\alpha_1)\dot{\alpha}_1^2}{(m_1 r^2  - m_1^2 r^2 l^2 cos^2(\alpha_1)/ (m_1 l^2 + I_1) + m_2 r^2 + I_2)} $

$\ddot{\alpha}_1  = (M -  m_1 r \ddot{\alpha}_2 l cos(\alpha_1) + lm_1gsin(\alpha_1)) / (m_1 l^2 + I_1)$

From these equations we can simulate the robot by either integrateing with a fixed time step or use a differential solver e.g. runge-kutta 45.  

## Data collection

Before we can do any regression on the problem we need to collect data. The data is collected while using a controller that follows a position reference signal. 


## Exercise 2:
For the first test we set the position reference signal to constant e.g $ \textit{pos_ref} = 0$. But there is one problem with using a constant reference signal. Discuss below why this should give a poor estimation of the parameters. 

Anwser:    Om man använder sig utav en konstant så kommer inte roboten att kunna läsa av bra data utan den kommer att läsa av några data tills den balacerar sig på den konstanten vilket gör att skattade data kommer att bli dåliga data. När man har en sinus signal så kommer den roboten att varira mellan minsta värde och högsta värde vilket medför att roboten måste balancera sig och därifrån får man bra data som ger bra skattade data.  

In [42]:


class Controller(object):
    def __init__(self, dt, init_states):
        self.dt = dt
        # controller parameter for position controller
        self.K_pos = 3
        self.D_pos = 0.5
        # controller parameter for angle controller
        self.K_ang = 10
        self.I_ang = 1
        self.D_ang = 1
        # limit the maximum allowed reference angle
        self.max_angle = 0.8
        # variables for the integrator in the PID controller
        self.angle_int = 0
        self.pre_o1 = init_states['a1']
    
    def update(self, t, states): 
        # The update function will be called on every timestep and return the torque for the motors. 
        # The position reference to 
        pos_ref = 0.2*np.sin(t)  # PD controller for position


        err = self.K_pos*(pos_ref - states['pos']) - self.D_pos * states['vel']
        # The error is used as referece angle, with upper and lower bounds. 
        ref_ang = -np.clip(err, -self.max_angle, self.max_angle)
        self.angle_int += self.dt * (self.pre_o1 - states['a1'])
        self.pre_o1 = states['a1']
        # PID controller for the angle, generates the desiered motor torque. 
        M = self.K_ang*(ref_ang-states['a1']) + self.I_ang*self.angle_int - self.D_ang*states['da1']
    
        return M


## Exercise 2:

In the code below we have the main loop. Your task is to save the data needed in places marked with TODO. 

In [51]:
# number of steps and step size to run the simulation/data collection

n_steps = 1000
step_time = 1e-2

# Initiate the robot
robot = BalanceRobot(dt=step_time)
# True parametes in the robot.
g = robot.p["g"] # gravity.
r = robot.p["r"] # radious of wheel.
m2 = robot.p["m2"] # mass of wheel.
I2 = robot.p["I2"] # moment of intertia of wheel.

# These parameters should only be used as ground truth
I1 = robot.p["I1"] # moment of intertia of main body.
m1 = robot.p["m1"] # mass of main body.
l =  robot.p["l"] # lenth from wheel hub to center of mass of main body. 

# Initate the controller 
state = robot.get_state()
controller = Controller(dt=step_time, init_states=state)

## : Initilize X^T matrix and Y vector to hold data

X_T={"a":[],
    "b":[],
    "c":[]
    ,"d":[]}

Y={"e":[]
        ,"f":[]}
## scale noise
scale_noise = 10

## Main loop 

for i in range(n_steps):
    # calculate time
    t = i*step_time
    # calculate torque 
    state = robot.get_state()
    M = controller.update(t, state)
    # step one time step for robot simulation
    dda1, dda2 = robot.step(torque=M)
    
    ## Add noice to data
    da1 = state['da1'] + np.random.normal(0.0, scale_noise*0.02) # d/dt alpha 1
    a1 = state['a1'] + np.random.normal(0.0, scale_noise*0.03) # alpha 1
    da2 = state['da2'] + np.random.normal(0.0, scale_noise*0.02) # d/dt alpha 2
    a2 = state['a2'] + np.random.normal(0.0, scale_noise*0.03) # alpha 2
    dda1 += np.random.normal(0.0, scale_noise*0.01) # d^2/dt^2 alpha 1
    dda2 += np.random.normal(0.0, scale_noise*0.01) # d^2/dt^2 alpha 2
    M += np.random.normal(0.0, scale_noise*0.05) # torque 
    # a=dda1
    # b=r*a2*cos(a1)-sin(a1)*g
    # c=0
    # d=0
    # e=r*cos(a1)*dda1-r*sin(a1)*da1*da1
    # f=r*r*dda2
    X_T["a"].append(dda1)                               #x1
    X_T["b"].append(r*dda2*np.cos(a1)-np.sin(a1)*g)             #x1
    X_T["c"].append(r*np.cos(a1)*dda1-r*np.sin(a1)*da1*da1)   #x1
    X_T["d"].append(r*r*dda2)                           #x1
    Y["e"].append(M)                                  #Y1
    Y["f"].append(-M-I2*dda2-m2*r*r*dda2)             #Y2



    
    
    ## : save the data needed for the regression in exercise 1. 
    
    
    
    
pygame.quit()   

## Exercise 3: Linear regression 

Use linear regression to estimate the parameters.  

In [52]:
#Anwser: 
#X = 2X3
#Y = 1X2 

XX_T=np.zeros((3,3))
XY=np.zeros((3,1))
x_t=[]
y=[]



for index in range(len(X_T["a"])):
    a = X_T["a"][index]
    b = X_T["b"][index]
    c = X_T["c"][index]
    d = X_T["d"][index]
    e = Y["e"][index]
    f = Y["f"][index]
    
    XX_T = XX_T +  np.array([[a*a ,  a*b    , 0],     
                    [a*b  , b*b+c*c , c*d ],
                    [0    ,  c*d    , d*d]])

    XY = XY + np.array([[a*e],[b*e+c*f],[d*f]])
    x_t.append(np.array([[a ,  b    , 0],
                    [0 , c , d ]])) 
    y.append(np.array([[e],[f]])) 

 



theta=np.linalg.inv(XX_T).dot(XY)
est_m1 = theta[2,0]
est_l =  theta[1,0]/est_m1
est_I1 = -est_m1*est_l*est_l+theta[0,0]

print('Estimated mass = ', np.round(est_m1, 3), ' and true  = ', robot.p['m1'])
print('Estimated length = ', np.round(est_l, 3), ' and true value = ', robot.p['l'])
print('Estimated moment of inertia = ', np.round(est_I1, 3), ' and true value = ', robot.p['I1'])



Estimated mass =  -1.645  and true  =  0.5
Estimated length =  0.002  and true value =  0.15
Estimated moment of inertia =  -0.028  and true value =  0.006


## Exercise 4: Calculate the cost function 

When solvning the linear regression problem we minimize the cost function. Calculate the value of the cost function on the saved data with the parameters estimated.

In [50]:

cost_value = 0
for index in range(len(x_t)):

    pred_y = np.dot(x_t[index], theta)
    diff = np.linalg.norm(pred_y - y[index])**2 
    cost_value += diff
    
cost_value = cost_value / len(x_t)
print('cost_value', cost_value)


cost_value 0.020772931794788563


## Exercise 5: Lets improve the identification  

Try to change the position reference signal in the controller, try a few different ones and report the identified parameters and loss value below together with the function used. Only the first line in the controller class in the update function need to be changed. A example could be: pos_ref = 0.2*np.sin(t). 


## Exercise 6: Analyse cost function

Why does the cost function value not improve drastically with the better identified parameters?

Anwser:

| ref signal | cost function value | $m_1$ | l| $I_1$ |
| --- | --- | --- | --- | --- |
| 0 |0.004977655507765656  |-2.118 | 0.003 | -0.011 | 
| 0.2*np.sin(t) | 0.005038524999713991 |0.476 | 0.049 | 0.017 | 
|  0.2*np.sin(10t)| 0.005696834328688252|0.436 |0.137  |  0.008| 
| 2*np.cos(t) |  0.005555041339476233| 0.495| 0.15 | 0.006 | 

Anwser:För att cost func påverkas inte av vart roboten ska till i position, utan modellen är redan tränad och pos_ref visar bara vart ska roboten balancera sig. 




 ## Exercise 7:  Noise

Explore how the noise levels added before the data collection affects the identification results and also the value of the cost function. Why is it that the cost function reacts differenlty compared to exersice 5? Use the best performing position reference signal. 

Anwser: För att bruset påverkar referenssignal där den gör att distansfelet mellan signalen och data blir hög. 
 The pos_ref: 0.2*np.sin(t)
| scale_noise | cost function value | $m_1$ | l| $I_1$ |
| --- | --- | --- | --- | --- |
| 0.1 |  5.822008121579131e-05|  0.49|0.146  |0.007 | 
| 0.5 |0.0012187412785607814 | 0.447  |  0.097| 0.014| 
| 1 | 0.005038524999713991 |0.476 | 0.049 | 0.017 |
| 2 |0.020772931794788563| 0.434 | 0.03 |0.018| 
| 10 |0.4977031371161539 | -1.645 | 0.002|-0.028| 


## Exersice 8: 

Using the intuition built from the above exercises, discuss what would be important if you where to use this method on a real world example. 

I verkligheter kan man inte förutse bruset och desutom är det inte normalfördelat vilket gör att det är svårt att kunna skatta, däremot i simuleringen är det ganska snälla brus och desutom normalfördelat vilket gör att skattningen blir lättare men inte nödvändigtvis i verkliga system.  I formeln tar man inte hänsyn till effekter såsom friktion. 